In [1]:
from dialogue2graph.pipelines.model_storage import ModelStorage
from dialogue2graph.pipelines.d2g_llm.pipeline import Pipeline

/home/askatasuna/Документы/DeepPavlov/chatsky-llm-autoconfig/.venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "validate" in "Dialogue" shadows an attribute in parent "BaseModel"
  warnings.warn(
INFO:datasets:PyTorch version 2.6.0 available.
